In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.layers import Dense,Dropout
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


In [30]:
df = pd.read_csv(r'https://raw.githubusercontent.com/kurianbenoy/DataminingLab/master/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [31]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [32]:
df.shape

(7043, 21)

In [33]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [38]:
df = df.drop('customerID',axis=1)
df.gender = df.gender.replace("Male", 1)
df.gender = df.gender.replace("Female", 0)

map_ = {'No': 0, 'Yes': 1}
df['Partner'] = df['Partner'].map(map_)
df['Dependents'] = df['Dependents'].map(map_)
df['PhoneService'] = df['PhoneService'].map(map_)
df['PaperlessBilling'] = df['PaperlessBilling'].map(map_)
df["Churn"] = df["Churn"].map(map_)

label = LabelEncoder()
df['InternetService'] = label.fit_transform(df['InternetService'])
df['Contract'] = label.fit_transform(df['Contract'])
df['PaymentMethod'] = label.fit_transform(df['PaymentMethod'])
df['MultipleLines'] = label.fit_transform(df['MultipleLines'])
df['OnlineSecurity'] = label.fit_transform(df['OnlineSecurity'])
df['OnlineBackup'] = label.fit_transform(df['OnlineBackup'])
df['DeviceProtection'] = label.fit_transform(df['DeviceProtection'])
df['TechSupport'] = label.fit_transform(df['TechSupport'])
df['StreamingTV'] = label.fit_transform(df['TechSupport'])
df['StreamingMovies'] = label.fit_transform(df['TechSupport'])

In [52]:
df.shape

(7043, 20)

In [48]:
df["TotalCharges"].value_counts()

          11
20.2      11
19.75      9
20.05      8
19.9       8
          ..
6849.4     1
692.35     1
130.15     1
3211.9     1
6844.5     1
Name: TotalCharges, Length: 6531, dtype: int64

In [53]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors='coerce')

In [54]:
df = df.dropna()

In [55]:
df.shape

(7032, 20)

In [56]:
df.dtypes

gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
PhoneService          int64
MultipleLines         int64
InternetService       int64
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
Contract              int64
PaperlessBilling      int64
PaymentMethod         int64
MonthlyCharges      float64
TotalCharges        float64
Churn                 int64
dtype: object

In [57]:
dataset = df.values
X = df.iloc[:, :-1]
Y = df.iloc[:,-1]

## Correct Cross validation

In [58]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X, Y, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [59]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

## Modelling

In [64]:
model=Sequential()
model.add(Dense(15,input_dim=19, activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))

In [65]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 15)                300       
                                                                 
 dense_9 (Dense)             (None, 10)                160       
                                                                 
 dense_10 (Dense)            (None, 8)                 88        
                                                                 
 dropout_2 (Dropout)         (None, 8)                 0         
                                                                 
 dense_11 (Dense)            (None, 1)                 9         
                                                                 
Total params: 557
Trainable params: 557
Non-trainable params: 0
_________________________________________________________________


In [66]:
model.compile(loss="binary_crossentropy", optimizer="SGD", metrics=['accuracy'])

In [67]:
model_history = model.fit(X_train, y_train, epochs=200, validation_data=(X_valid, y_valid))

Epoch 1/200
124/124 [==============================] - 1s 3ms/step - loss: 0.6383 - accuracy: 0.7145 - val_loss: 0.6038 - val_accuracy: 0.7369
Epoch 2/200
124/124 [==============================] - 0s 2ms/step - loss: 0.5935 - accuracy: 0.7302 - val_loss: 0.5669 - val_accuracy: 0.7384
Epoch 3/200
124/124 [==============================] - 0s 2ms/step - loss: 0.5713 - accuracy: 0.7302 - val_loss: 0.5464 - val_accuracy: 0.7384
Epoch 4/200
124/124 [==============================] - 0s 2ms/step - loss: 0.5534 - accuracy: 0.7305 - val_loss: 0.5322 - val_accuracy: 0.7384
Epoch 5/200
124/124 [==============================] - 0s 2ms/step - loss: 0.5430 - accuracy: 0.7305 - val_loss: 0.5201 - val_accuracy: 0.7384
Epoch 6/200
124/124 [==============================] - 0s 2ms/step - loss: 0.5301 - accuracy: 0.7305 - val_loss: 0.5074 - val_accuracy: 0.7384
Epoch 7/200
124/124 [==============================] - 0s 2ms/step - loss: 0.5198 - accuracy: 0.7305 - val_loss: 0.4967 - val_accuracy: 0.7384

In [68]:
model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=['accuracy'])
model_history = model.fit(X_train, y_train, epochs=200, validation_data=(X_valid, y_valid))

Epoch 1/200
124/124 [==============================] - 1s 3ms/step - loss: 0.4095 - accuracy: 0.8023 - val_loss: 0.4239 - val_accuracy: 0.8006
Epoch 2/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4059 - accuracy: 0.8048 - val_loss: 0.4245 - val_accuracy: 0.8029
Epoch 3/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4074 - accuracy: 0.8056 - val_loss: 0.4245 - val_accuracy: 0.7991
Epoch 4/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4041 - accuracy: 0.8046 - val_loss: 0.4240 - val_accuracy: 0.8052
Epoch 5/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4075 - accuracy: 0.8073 - val_loss: 0.4242 - val_accuracy: 0.8006
Epoch 6/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4049 - accuracy: 0.8101 - val_loss: 0.4244 - val_accuracy: 0.8021
Epoch 7/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4020 - accuracy: 0.8086 - val_loss: 0.4271 - val_accuracy: 0.7983

In [69]:
model.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=['accuracy'])
model_history = model.fit(X_train, y_train, epochs=200, validation_data=(X_valid, y_valid))

Epoch 1/200
124/124 [==============================] - 1s 3ms/step - loss: 0.3473 - accuracy: 0.8346 - val_loss: 0.6303 - val_accuracy: 0.7945
Epoch 2/200
124/124 [==============================] - 0s 2ms/step - loss: 0.3470 - accuracy: 0.8273 - val_loss: 0.6270 - val_accuracy: 0.7854
Epoch 3/200
124/124 [==============================] - 0s 2ms/step - loss: 0.3504 - accuracy: 0.8288 - val_loss: 0.6394 - val_accuracy: 0.7900
Epoch 4/200
124/124 [==============================] - 0s 2ms/step - loss: 0.3445 - accuracy: 0.8306 - val_loss: 0.6462 - val_accuracy: 0.7945
Epoch 5/200
124/124 [==============================] - 0s 2ms/step - loss: 0.3490 - accuracy: 0.8263 - val_loss: 0.6328 - val_accuracy: 0.7938
Epoch 6/200
124/124 [==============================] - 0s 2ms/step - loss: 0.3466 - accuracy: 0.8329 - val_loss: 0.6270 - val_accuracy: 0.7908
Epoch 7/200
124/124 [==============================] - 0s 2ms/step - loss: 0.3445 - accuracy: 0.8339 - val_loss: 0.6404 - val_accuracy: 0.7983